In [ ]:
## Q3. "Plausibly Exogenous"

In [ ]:
### 1. How can instrumental variables Z help identify Beta in the cannonical Wright 1934 model? 

In [ ]:
Say we have identified an instrument Z that satisfies "E(u'Z) = 0." 